In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings 
warnings.filterwarnings("ignore")

from dataset import train_test_dfs, preprocess 
from rmodels import RModels
from emodels import EModels
from nn_model import NN, NNP

import os
import copy

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Choose best models to select ideal values for parameter : a & b

In [2]:
path = "D:\sulf\CLRF"
lst_files = os.listdir(path)
print("Total number of predictions: ", len(lst_files))

dfile = [f for f in lst_files if "DF_DD" in f and f.endswith('.xlsx')]
print(len(dfile))

Total number of predictions:  8356
521


In [3]:
dfile.pop(373)

'DF_DD_2478.7702925768044-182.12522493010147.xlsx'

In [4]:
r2_s = []; 
for i, df in enumerate(dfile):
    data = pd.read_excel(path + "\\" + df)
    data.drop("Unnamed: 0", axis=1, inplace=True)
    r2_s.append(r2_score(data["SCSS"], data["CLRF"]))
    
ind = np.argsort(r2_s)[::-1]
for i in ind:
    print(dfile[i])

DF_DD_-376.8496453124899-44.153749533618566.xlsx
DF_DD_551.2275770453762--121.83941511007811.xlsx
DF_DD_-515.2674060969039--14.204682985502245.xlsx
DF_DD_-693.2155216191641--19.328530921533684.xlsx
DF_DD_-450.0594924932302-153.35117941641738.xlsx
DF_DD_649.2507510546802--437.5838929750024.xlsx
DF_DD_772.8894899320658--136.91728562977937.xlsx
DF_DD_728.3148378287315--445.7395237863342.xlsx
DF_DD_569.628476855476--496.2947665921429.xlsx
DF_DD_-753.3257037388194--40.56162685161996.xlsx
DF_DD_859.9898531473409--416.8278329232289.xlsx
DF_DD_108.96872390888893--418.0704885759865.xlsx
DF_DD_31.751418491747245--436.83466282419056.xlsx
DF_DD_-208.0822715413924--347.9258250403939.xlsx
DF_DD_534.1963479352071--756.5294393099538.xlsx
DF_DD_-790.1906438968417-191.65959913272536.xlsx
DF_DD_-166.61329991166895--449.94640558210494.xlsx
DF_DD_844.8314784335889--826.100627311318.xlsx
DF_DD_-506.50950781085885--336.3838523339009.xlsx
DF_DD_601.2419607427564--887.0622908611346.xlsx
DF_DD_1174.147164797776

In [5]:
a = 551.22; b = -121.83 # We choose the 2nd best model

# Hybrid Models for Datasets : 1, 2, 3 & 4

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings 
warnings.filterwarnings("ignore")

from dataset import train_test_dfs, preprocess 
from rmodels import RModels
from emodels import EModels
from nn_model import NN, NNP

import os
"""
prediction results for final_files folder in sulf
"""
path = r"D:\sulf\final_files"
files = os.listdir(path)
print(files)

['Dataset #1.xlsx', 'Dataset #2.xlsx', 'Dataset #3.xlsx', 'Dataset #4.xlsx']


In [8]:
df_names = ["Dataset 1", "Dataset 2", "Dataset 3", "Dataset 4"]
df, df_scaled = {}, {}
for f, names in zip(files, df_names):
    data = pd.read_excel(path + "\\" + f, engine="openpyxl")
    data.fillna(0, inplace=True)
    data.drop(columns=["Unnamed: 17", "Hybrid model", "Label for train and test"], axis=1, inplace = True)
    data.rename(columns={"SCSS Measurement":"SCSS"}, inplace=True)
    df[names] = data

In [9]:
import copy

'''
ONLY PREDICTS RESULTS FOR THE HYBRID MODEL
'''

pred_path = r"D:\sulf\final_files_pred"
a = 551.22; b = -121.83
pred_cols = ['Pressure', 'T', 'SiO2', 'TiO2', 'Al2O3', 'FeO', 'MgO', 'CaO', 'NiO',
       'Na2O', 'K2O', 'H2O', 'Fe', 'Ni+Cu+Co', 'S', 'O']
e_names = ["XGB", "RF", "ET", "ADA", "DT", "SVR"]

# r_names = ["LR", "RR", "POLY", "PLS", "KNN", "BR"]
# n_names = ["ANN"]
for k, n in enumerate(df_names):
    d = copy.deepcopy(df[n])
    x_train, x_test, y_train, y_test, scaler = train_test_dfs(d, test_size=0.30)
    x_tr, x_ts, y_tr, y_ts = train_test_dfs(d, test_size=0.30, scale=False)
    train_idx = list(x_tr.index)
    emp = EModels(x_train, x_test, y_train, y_test)
#     nnp = NNP(x_train, x_test, y_train, y_test, df, d[pred_cols].shape[1], activation = 'relu', lr = 0.001)
#     rmp = RModels(x_train, x_test, y_train, y_test)

    _, y_pred_emp = emp.predict_em(scaler.transform(d[pred_cols]))
#     y_pred_nnp = nnp.predict_nn(scaler.transform(df[pred_cols]))
#     y_pred_rmp = rmp.predict_rm(scaler.transform(df[pred_cols]), optimize=False)

    for e_id, e_name in enumerate(e_names):
        df[n][e_name] = y_pred_emp[e_id]

#     for n_id, n_name in enumerate(n_names):
#         df4[n_name] = y_pred_nnp

#     for r_id, r_name in enumerate(r_names):
#         df4[r_name] = y_pred_emp[r_id]

    df[n]["CLRF"] = np.exp(a/df[n]["T"] + b*df[n]["Pressure"]/df[n]["T"]) * df[n]["RF"]
    df[n]["train label"] = 0
    for i in list(df[n].index):
        if i in train_idx:
            df[n].loc[i, ["train label"]] = 1
    df[n].to_excel(pred_path + "\\" + n + "_{}-{}".format(a,b) + ".xlsx")

[02:21:06] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "boost_params" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[02:21:13] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "boost_params" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[02:21:19] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "boost_params" } might not be used.

  Thi

# Predictions on other datasets trained on Dataset#4

In [14]:
df = pd.read_excel("Dataset #4.xlsx")
df.rename(columns={"SCSS measurement":"SCSS"}, inplace=True)
df.fillna(0, inplace=True)
df.head()

,Pressure,T,SiO2,TiO2,Al2O3,FeO,MgO,CaO,NiO,Na2O,K2O,H2O,Fe,Ni+Cu+Co,S,O,SCSS
0,2.5,1773.0,36.83,10.56,9.94,18.57,9.24,12.89,0.43,0.34,0.03,0.0,64.06,0.0,34.19,1.64,3834.0
1,2.5,1773.0,41.24,4.84,12.44,15.94,7.58,13.37,0.01,0.67,0.14,0.0,64.74,0.0,33.81,1.49,2490.0
2,2.5,1673.0,33.28,17.21,7.34,22.97,5.90,10.85,0.18,0.36,0.07,0.0,63.94,0.0,34.07,1.76,4570.0
3,2.0,1773.0,38.52,9.87,9.93,18.05,8.38,12.30,0.33,0.34,0.07,0.0,63.59,0.0,33.27,2.32,4658.0
4,1.5,1873.0,42.89,4.77,13.35,17.69,6.76,11.79,0.01,0.49,0.14,0.0,63.90,0.0,33.78,2.80,4798.0


In [15]:
path = r"D:\sulf\Latest EMP"
files = os.listdir(path)
print(files)

['Earth_Komatiite_IT.xlsx', 'Earth_MORB_LT.xlsx', 'Earth_Pyrolite_HT.xlsx', 'Earth_Pyrolite_IT.xlsx', 'Earth_Pyrolite_LT.xlsx', 'high.xlsx', 'low.xlsx', 'Mars_50%_Dry_IT.xlsx', 'Mars_50%_Wet_IT.xlsx', 'Mars_basalt_Dry_LT.xlsx', 'Mars_basalt_Wet_LT.xlsx', 'Mars_Bulk_Dry_HT.xlsx', 'Mars_Bulk_Wet_HT.xlsx', 'mid.xlsx', 'Moon_Longhi 50%_IT.xlsx', 'Moon_Longhi.xlsx', 'Moon_Taylor 50%_IT.xlsx']


In [16]:
for f in files:
    d_col = pd.read_excel(path + "//" + f)
    print(d_col.columns.values)

['Pressure' 'T' 'SiO2' 'TiO2' 'Al2O3' 'FeO' 'MgO' 'CaO' 'NiO' 'Na2O' 'K2O'
 'H2O' 'Fe' 'Ni+Cu+Co' 'S' 'O']
['Pressure' 'T' 'SiO2' 'TiO2' 'Al2O3' 'FeO' 'MgO' 'CaO' 'NiO' 'Na2O' 'K2O'
 'H2O' 'Fe' 'Ni+Cu+Co' 'S' 'O']
['Pressure' 'T' 'SiO2' 'TiO2' 'Al2O3' 'FeO' 'MgO' 'CaO' 'NiO' 'Na2O' 'K2O'
 'H2O' 'Fe' 'Ni+Cu+Co' 'S' 'O']
['Pressure' 'T' 'SiO2' 'TiO2' 'Al2O3' 'FeO' 'MgO' 'CaO' 'NiO' 'Na2O' 'K2O'
 'H2O' 'Fe' 'Ni+Cu+Co' 'S' 'O']
['Pressure' 'T' 'SiO2' 'TiO2' 'Al2O3' 'FeO' 'MgO' 'CaO' 'NiO' 'Na2O' 'K2O'
 'H2O' 'Fe' 'Ni+Cu+Co' 'S' 'O']
['Pressure' 'T' 'SiO2' 'Al2O3' 'FeO' 'MgO' 'CaO' 'NiO' 'NaO' 'K2O' 'H2O'
 'Fe' 'Ni+Cu+Co' 'S' 'O']
['Unnamed: 0' 'Pressure' 'T' 'SiO2' 'Al2O3' 'FeO' 'MgO' 'CaO' 'NiO' 'NaO'
 'K2O' 'H2O' 'Fe' 'Ni+Cu+Co' 'S' 'O']
['Pressure' 'T' 'SiO2' 'TiO2' 'Al2O3' 'FeO' 'MgO' 'CaO' 'NiO' 'Na2O' 'K2O'
 'H2O' 'Fe' 'Ni+Cu+Co' 'S' 'O']
['Pressure' 'T' 'SiO2' 'TiO2' 'Al2O3' 'FeO' 'MgO' 'CaO' 'NiO' 'Na2O' 'K2O'
 'H2O' 'Fe' 'Ni+Cu+Co' 'S' 'O']
['Pressure' 'T' 'SiO2' 'TiO2' 'Al2O3' 'F

In [17]:
pred_path = r"D:\sulf\EMP Pred Revised"
cols = ['Pressure' 'T' 'SiO2' 'TiO2' 'Al2O3' 'FeO' 'MgO' 'CaO' 'NiO' 'Na2O' 'K2O'
 'H2O' 'Fe' 'Ni+Cu+Co' 'S' 'O']
e_names = ["XGB", "RF", "ET", "ADA", "DT", "SVR"]
for f in files:
    a = 551.22; b = -121.83
    d = pd.read_excel(path + "//" + f)
    if "Unnamed: 0" in d.columns.values.tolist():
        d.drop(columns = ["Unnamed: 0"], inplace=True)
    if "Hybrid Model" in d.columns.values.tolist():
        d.drop(columns = ["Hybrid Model"], inplace=True)
    #d.rename(columns={'P':"Pressure", "NiO*":"NiO"}, inplace=True)
    d.rename(columns={"NaO": "Na2O"}, inplace=True)
    col = d.columns.values.tolist()
    df_mod = df[col + ["SCSS"]]
    dfp = copy.deepcopy(d)
    x_train, x_test, y_train, y_test, scaler = train_test_dfs(df_mod, test_size=.30)
    emp = EModels(x_train, x_test, y_train, y_test)
    _, y_pred_emp = emp.predict_em(scaler.transform(dfp))

    for e_id, e_name in enumerate(e_names):
        d[e_name] = y_pred_emp[e_id]

    d["CLRF"] = np.exp(a/d["T"] + b*d["Pressure"]/d["T"]) * d["RF"]
    d.to_excel(pred_path + "\\" + "Pred_" + f)

[02:33:40] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "boost_params" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[02:33:49] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "boost_params" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[02:33:57] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "boost_params" } might not be used.

  Thi

In [2]:
df = pd.read_excel("Dataset #4.xlsx")
df.rename(columns={"SCSS measurement":"SCSS"}, inplace=True)
df.fillna(0, inplace=True)
df.head()

,Pressure,T,SiO2,TiO2,Al2O3,FeO,MgO,CaO,NiO,Na2O,K2O,H2O,Fe,Ni+Cu+Co,S,O,SCSS
0,2.5,1773.0,36.83,10.56,9.94,18.57,9.24,12.89,0.43,0.34,0.03,0.0,64.06,0.0,34.19,1.64,3834.0
1,2.5,1773.0,41.24,4.84,12.44,15.94,7.58,13.37,0.01,0.67,0.14,0.0,64.74,0.0,33.81,1.49,2490.0
2,2.5,1673.0,33.28,17.21,7.34,22.97,5.90,10.85,0.18,0.36,0.07,0.0,63.94,0.0,34.07,1.76,4570.0
3,2.0,1773.0,38.52,9.87,9.93,18.05,8.38,12.30,0.33,0.34,0.07,0.0,63.59,0.0,33.27,2.32,4658.0
4,1.5,1873.0,42.89,4.77,13.35,17.69,6.76,11.79,0.01,0.49,0.14,0.0,63.90,0.0,33.78,2.80,4798.0


In [18]:
tdf = pd.read_excel("Steenstra.xlsx")
tdf.head()

,Pressure,T,SiO2,TiO2,Al2O3,FeO,MgO,CaO,NiO,Na2O,K2O,H2O,Fe,Ni+Cu+Co,S,O
0,53,4605,44.25295,1.867359,18.81181,12.01483,8.756995,8.073137,0.890826,1.711025,0.135192,0.0001,55.9,1.4,26.3,6.2
1,43,4300,36.01825,1.439235,22.63040,18.04694,8.482422,6.505754,1.178425,1.581727,0.065561,0.0001,55.9,1.4,26.3,6.2
2,44,3927,34.00400,2.321821,13.14647,26.53738,7.693807,6.845227,0.016120,1.921718,0.144116,0.0001,55.9,1.4,26.3,6.2
3,53,4605,44.25295,1.867359,18.81181,12.01483,8.756995,8.073137,0.890826,1.711025,0.135192,0.0001,54.5,10.8,34.0,0.7
4,43,4300,36.01825,1.439235,22.63040,18.04694,8.482422,6.505754,1.178425,1.581727,0.065561,0.0001,54.5,10.8,34.0,0.7


In [19]:
tdf.rename(columns={'P':'Pressure', 'NiO*':'NiO', 'NaO':'Na2O', 'Ni':'Ni+Cu+Co'}, inplace=True)
tdf.head()

,Pressure,T,SiO2,TiO2,Al2O3,FeO,MgO,CaO,NiO,Na2O,K2O,H2O,Fe,Ni+Cu+Co,S,O
0,53,4605,44.25295,1.867359,18.81181,12.01483,8.756995,8.073137,0.890826,1.711025,0.135192,0.0001,55.9,1.4,26.3,6.2
1,43,4300,36.01825,1.439235,22.63040,18.04694,8.482422,6.505754,1.178425,1.581727,0.065561,0.0001,55.9,1.4,26.3,6.2
2,44,3927,34.00400,2.321821,13.14647,26.53738,7.693807,6.845227,0.016120,1.921718,0.144116,0.0001,55.9,1.4,26.3,6.2
3,53,4605,44.25295,1.867359,18.81181,12.01483,8.756995,8.073137,0.890826,1.711025,0.135192,0.0001,54.5,10.8,34.0,0.7
4,43,4300,36.01825,1.439235,22.63040,18.04694,8.482422,6.505754,1.178425,1.581727,0.065561,0.0001,54.5,10.8,34.0,0.7


In [20]:
cols = ['Pressure', 'T', 'SiO2', 'TiO2', 'Al2O3', 'FeO', 'MgO', 'CaO', 'NiO', 'Na2O', 'K2O',
 'H2O', 'Fe', 'Ni+Cu+Co', 'S','O']
for col in cols:
    tdf[col] = tdf[col].astype('float')

In [21]:
e_names = ["XGB", "RF", "ET", "ADA", "DT", "SVR"]
a = 551.22; b = -121.83 # parameter prediction from best model
x_train, x_test, y_train, y_test, scaler = train_test_dfs(df, test_size=.30)
emp = EModels(x_train, x_test, y_train, y_test)
_, y_pred_emp = emp.predict_em(scaler.transform(tdf))
print(tdf.head())
for e_id, e_name in enumerate(e_names):
    tdf[e_name] = y_pred_emp[e_id]
tdf["CLRF"] = np.array(np.exp(a/tdf["T"] + b*tdf["Pressure"]/tdf["T"]) * tdf["RF"], dtype=np.float32)
tdf.to_excel("Steenstra_Predicted.xlsx")

[02:39:44] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "boost_params" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


   Pressure       T      SiO2      TiO2     Al2O3       FeO       MgO  \
0      53.0  4605.0  44.25295  1.867359  18.81181  12.01483  8.756995   
1      43.0  4300.0  36.01825  1.439235  22.63040  18.04694  8.482422   
2      44.0  3927.0  34.00400  2.321821  13.14647  26.53738  7.693807   
3      53.0  4605.0  44.25295  1.867359  18.81181  12.01483  8.756995   
4      43.0  4300.0  36.01825  1.439235  22.63040  18.04694  8.482422   

        CaO       NiO      Na2O       K2O     H2O    Fe  Ni+Cu+Co     S    O  
0  8.073137  0.890826  1.711025  0.135192  0.0001  55.9      